In [69]:
import spotipy
import spotipy.util as util
from pprint import pprint
import json
import argparse
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

client_id = 'a4601a5ae121413e9450015d6c0febaa'
client_secret = 'ed18559d5243412281ab9c5e80a0db65'
username='5dpgk6tgb1w0swe0ni3v8s1c9'
redirect_uri='http://localhost:3000/'


token = util.prompt_for_user_token(
        username=username,
        scope=['user-read-recently-played', 'playlist-read-private'],
        client_id=client_id,
        client_secret=client_secret,
        redirect_uri=redirect_uri)

sp = spotipy.Spotify(auth=token)

In [31]:
def get_playlist_info(username, playlist_uri):
    # initialize vars
    offset = 0
    tracks, uris, names, artists = [], [], [], []

    # get playlist id and name from URI
    playlist_id = playlist_uri.split(':')[2]
    playlist_name = sp.user_playlist(username, playlist_id)['name']

    # get all tracks in given playlist (max limit is 100 at a time --> use offset)
    while True:
        results = sp.user_playlist_tracks(username, playlist_id, offset=offset)
        tracks += results['items']
        if results['next'] is not None:
            offset += 100
        else:
            break
        
    # get track metadata
    for track in tracks:
        try:
            names.append(track['track']['name'])
            artists.append(track['track']['artists'][0]['name'])
            uris.append(track['track']['uri'])
        except:
            pass
    
    return playlist_name, names, artists, uris


In [3]:
# Extract features from each track in a playlist
def get_features_for_playlist(df, username, uri):
  
    # get all track metadata from given playlist
    playlist_name, names, artists, uris = get_playlist_info(username, uri)
    
    # iterate through each track to get audio features and save data into dataframe
    for name, artist, track_uri in zip(names, artists, uris):
        
        # access audio features for given track URI via spotipy 
        audio_features = sp.audio_features(track_uri)

        # get relevant audio features
        feature_subset = [audio_features[0][col] for col in df.columns if col not in ["name", "artist", "track_URI", "playlist"]]

        # compose a row of the dataframe by flattening the list of audio features
        row = [name, artist, track_uri, *feature_subset, playlist_name]
        df.loc[len(df.index)] = row
    return df

In [20]:
def get_all_songs(df, user, playlist_id):
    df = pd.DataFrame(columns=['name', 'artist', 'track_URI', 'acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'valence', 'playlist'])
    for id in playlist_id:
        df = get_features_for_playlist(df, user, id).drop_duplicates()

    return df

In [24]:
happy_playlist = [
    'spotify:playlist:37i9dQZF1DX3rxVfibe1L0', 
    'spotify:playlist:37i9dQZF1DX6GwdWRQMQpq', 
    'spotify:playlist:37i9dQZF1DX66m4icL86Ru', 
    'spotify:playlist:0RH319xCjeU8VyTSqCF6M4',
    'spotify:playlist:37i9dQZF1DWZKuerrwoAGz']
happy1 = pd.DataFrame(columns=['name', 'artist', 'track_URI', 'acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'valence', 'playlist'])
happy1 = get_all_songs(happy1, username, happy_playlist)

In [32]:
sad_playlist = [
    'spotify:playlist:37i9dQZF1DX3YSRoSdA634',
    'spotify:playlist:37i9dQZF1DWSqBruwoIXkA',
    'spotify:playlist:4yXfnhz0BReoVfwwYRtPBm'
]
sad = pd.DataFrame(columns=['name', 'artist', 'track_URI', 'acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'valence', 'playlist'])
sad = get_all_songs(sad, username, sad_playlist)

In [70]:
mixed_playlist = ['spotify:playlist:4Yx4qIVyn4A1Hn5T2DVA3P']
mixed = pd.DataFrame(columns=['name', 'artist', 'track_URI', 'acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'valence', 'playlist'])
mixed = get_all_songs(mixed, username, mixed_playlist)

In [41]:
ones = np.ones(554)
happy1.insert(3, 'label', ones)

zeros = np.zeros(586)
sad.insert(3, 'label', zeros)

In [46]:
full_data = happy1.merge(sad, how='outer')

In [57]:
train_full = full_data.drop(['name', 'artist', 'track_URI', 'playlist'], axis=1)

In [58]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(train_full.drop('label', axis=1), train_full['label'], test_size=0.2, random_state=0)

In [60]:
from sklearn.model_selection import cross_val_score
lr = LogisticRegression()
lr.fit(x_train, y_train)
np.mean(cross_val_score(lr, x_test, y_test))

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.8901449275362319

In [72]:
vnpred = lr.predict_proba(mixed.drop(['name', 'artist', 'track_URI', 'playlist'], axis=1))[:,1]
mixed.insert(3, 'prediction', vnpred)

In [88]:
y = range(5)
q = map(lambda x: x*x, y)

In [92]:
for i in range(len(mixed)):
    print(mixed.loc[i, 'name'], "  ", mixed.loc[i, 'prediction'])

Trốn Tìm    0.03546764023772205
Có Lẽ Anh Chưa Từng    0.21574104916474918
Trên Tình Bạn Dưới Tình Yêu    0.9598039152042963
Buồn Thì Cứ Khóc Đi    0.3965703357408827
từ chối nhẹ nhàng thôi    0.3930356469731377
Lối Nhỏ    0.0879273677094255
Bài Này Chill Phết    0.644691679680936
Thanh Xuan    0.03481147722774211
Một Cú Lừa    0.6150418584193801
em bỏ hút thuốc chưa?    0.046746688444983565
Ke Cap Gap Ba Gia (feat. BINZ)    0.3349275954731693
Duyen Am    0.5471182690000629
Em Không Sai, Chúng Ta Sai    0.0736654537705942
Thang Dien    0.3964619883390654
Da Lo Yeu Em Nhieu    0.07074671209581918
Mặt Trời Của Em    0.3852753155336503
Ngày Mai Em Đi    0.5115372648223231
Vai Lan Don Dua    0.5506585888989752
Nước Mắt Em Lau Bằng Tình Yêu Mới    0.1469074613910439
Dấu Mưa    0.02525085858066182
Chiều Nay Không Có Mưa Bay    0.06277035761078842
Lạ Lùng    0.0078014901147104845
Hai Triệu Năm    0.6742779335471253
Anh Đếch Cần Gì Nhiều Ngoài Em    0.04504721468014283
Cho Tôi Lang Thang    0.

In [74]:
import pickle
pickle.dump(lr, open("deploy_model.sav", 'wb'))